In [40]:
import pandas as pd
#import library

In [18]:
sales = pd.read_csv(r'C:\Users\Acer\Downloads\data files and problem statement\Sales (1).tsv',sep="\t",header=0)
complaints = pd.read_csv(r'C:\Users\Acer\Downloads\data files and problem statement\Complaints (1).tsv',sep="\t",header=0)
Production_logs=pd.read_csv(r'C:\Users\Acer\Downloads\data files and problem statement\Production_logs (1).tsv',sep="\t",header=0)
#reading data as dataframes

In [19]:
sales = sales.iloc[0:,0:4]
Production_logs = Production_logs.iloc[0:,0:4]
#taking only important columns into consideration

# solution for report1

In [84]:
merged_Frame = pd.merge(sales,complaints, on = 'Invoice_Id',how='inner')
#merging columns sales and complaints based on invoice id as it only similar column

In [86]:
merged_Frame1 = pd.merge(merged_Frame,Production_logs, on = 'Batch_Id',how='inner')
#merging columns previous frame and Production_logs based on Batch_Id as it only similar column

In [126]:
df1=merged_Frame1['Items_Summary'].map(eval)
df1= df1.apply(pd.Series)
df1=df1.fillna(0)
df1=df1.rename(columns={"rock": "rock_Items_Summary", "scissor": "scissor_Items_Summary","paper": "paper_Items_Summary"})
#using map to convert string to dictionary
#seperating dictionaries
#filling the nan with zeroes
#changing name of the for easy understanding

In [128]:
df2=merged_Frame1['Items_produced'].map(eval)
df2= df2.apply(pd.Series)
df2=df2.fillna(0)
df2=df2.rename(columns={"rock": "rock_Items_produced", "scissor": "scissor_Items_produced","paper": "paper_Items_produced"})
#using map to convert string to dictionary
#seperating dictionaries
#filling the nan with zeroes
#changing name of the for easy understanding

In [130]:
df3=merged_Frame1['Items_discarded'].map(eval)
df3= df3.apply(pd.Series)
df3=df3.fillna(0)
df3=df3.rename(columns={"rock": "rock_Items_discarded", "scissor": "scissor_Items_discarded","paper": "paper_Items_discarded"})
#using map to convert string to dictionary
#seperating dictionaries
#filling the nan with zeroes
#changing name of the for easy understanding

In [156]:
df4=df.sort_values(by='Production_Unit_Id')
#sorting values based on production id 

In [157]:
df_main=df4.drop(['Items_Summary', 'Items_produced','Items_discarded'], axis=1)
#dropping unrequired columns

In [210]:
df_main_1=df_main.iloc[0:,3:]
df_main_2=df_main.iloc[0:,3]
df_main_3=df_main.iloc[0:,1]
#taking columns which are required

In [211]:
df_main_2=pd.get_dummies(df_main_2)
#converting complaints columns i.e categorical into numerical for further use

In [212]:
df_main_prob1 =pd.concat([df_main_3,df_main_2,df_main_1],join='outer',axis=1)
#merging the above frames 

In [213]:
df_main_prob1=df_main_prob1.drop(['Defective_Item'],axis=1)
#removing the column defective item as it is already converted to numerical in previous steps

In [215]:
df_main_prob1["final_paper_Items_Summary"] = df_main_prob1["paper"] * df_main_prob1["paper_Items_Summary"]
df_main_prob1["final_rock_Items_Summary"] = df_main_prob1["rock"] * df_main_prob1["rock_Items_Summary"]
df_main_prob1["final_scissor_Items_Summary"] = df_main_prob1["scissor"] * df_main_prob1["scissor_Items_Summary"]
#the numerical column is multiplied by items summary to get the amount of items which are complained 

In [217]:
df_main_prob1=df_main_prob1.iloc[0:,4:]
#taking the required columns

In [219]:
df_main_prob1=df_main_prob1.iloc[0:,4:]
#taking the required columns

In [220]:
df_main_prob1=pd.concat([df_main['Production_Unit_Id'],df_main_prob1],join='outer',axis=1)
#merging above frames as it does not have column names production id

In [222]:
df_main_prob1["final_rock_Total_defective"] = df_main_prob1["rock_Items_discarded"] + df_main_prob1["final_rock_Items_Summary"]
df_main_prob1["final_scissor_Total_defective"] = df_main_prob1["scissor_Items_discarded"] + df_main_prob1["final_scissor_Items_Summary"]
df_main_prob1["final_paper_Total_defective"] = df_main_prob1["paper_Items_discarded"] + df_main_prob1["final_paper_Items_Summary"]
#total defective is nothing but number of items discarded plus the items which are complained 

In [237]:
df_prob1=df_main_prob1.iloc[0:,0:4] 
df_prob2=df_main_prob1.iloc[0:,10:]
#taking required columns

In [239]:
df_prob=pd.concat([df_prob1,df_prob2],join='outer',axis=1)
#merging two frames to get features from both frames

In [254]:
df_prob=pd.concat([df_main['Batch_Id'],df_prob],join='outer',axis=1)
#merging two frames to get features from both frames because it doesnt have column named batch id

In [258]:
df_prob_1_prod=df_prob.iloc[0:,0:5]
df_prob_1_def=df_prob.iloc[0:,5:]
#taking required columns

In [260]:
df_prob_1_def=pd.concat([df_prob['Production_Unit_Id'],df_prob_1_def],join='outer',axis=1)
#merging two frames to get features from both frames because it doesnt have column named Production_Unit_Id

In [265]:
df_prob_1_prod_fin=df_prob_1_prod.drop_duplicates(subset=['Batch_Id'], keep='first')
#removing the duplicate items using batch id

In [268]:
df_prob_1_def_fin =df_prob_1_def.groupby('Production_Unit_Id').sum()     #total  defective for production id

In [273]:
df_prob_1_prod_fin=df_prob_1_prod_fin.groupby('Production_Unit_Id').sum() #total  produced for production id

In [287]:
df= pd.merge(df_prob_1_prod_fin,df_prob_1_def_fin, on = 'Production_Unit_Id',how='inner')
#merging two frames to get features from both frames because it doesnt have column named Production_Unit_Id

In [289]:
df["percentage_of_paper_defective"] = df["final_paper_Total_defective"] / df["paper_Items_produced"] * 100
df["percentage_of_rock_defective"] = df["final_rock_Total_defective"] / df["rock_Items_produced"] * 100
df["percentage_of_scissor_defective"] = df["final_scissor_Total_defective"] / df["scissor_Items_produced"] * 100
#percentage of defective is nothing but total defective divide by total produced which is multiplies by 100 

In [308]:
df_prob1_sol=df.drop(['final_rock_Total_defective','final_scissor_Total_defective','final_paper_Total_defective','scissor_Items_produced','paper_Items_produced','rock_Items_produced'], axis = 1)
#droping unrequired columns

In [313]:
df_prob1_sol.to_csv(r'C:\Users\Acer\Downloads\report1.csv')
#saving the data frame as csv 

# solution for report3

In [332]:
df_main_prob3=df_main_prob1.iloc[0:,0:7]
#taking required columns

In [333]:
df_main_prob3=pd.concat([df_main['Batch_Id'],df_main_prob3],join='outer',axis=1)
#merging two frames onn batch id to get common features

In [334]:
df_main_prob3=df_main_prob3.drop_duplicates(subset=['Batch_Id'], keep='first')
#removing duplicated i.e which is the repeated batch id rows

In [335]:
df_main_prob3=df_main_prob3.groupby('Production_Unit_Id').sum() 
#grouping the columns based on production id and summing up other columns

In [336]:
df_main_prob3['total_items_produced']=df_main_prob3['scissor_Items_produced']+df_main_prob3['paper_Items_produced']+df_main_prob3['rock_Items_produced']
df_main_prob3['total_items_discarded']=df_main_prob3['scissor_Items_discarded']+df_main_prob3['paper_Items_discarded']+df_main_prob3['rock_Items_discarded']
#merging columns it between i.e rock+paper+scissor=total for both produced and the items dicarded after the quality check

In [337]:
df_main_prob3=df_main_prob3.drop(['scissor_Items_produced','paper_Items_produced','rock_Items_produced','scissor_Items_discarded','paper_Items_discarded','rock_Items_discarded'],axis=1)
#removing the unrequired columns

In [339]:
df_main_prob3['%_detected_defects_by_QA']=df_main_prob3['total_items_discarded']/df_main_prob3['total_items_produced']*100
#total percentage of defective is equal to total items dicarded during QC by total items produced multiplied by 100

In [341]:
df_main_prob3=df_main_prob3.drop(['total_items_produced','total_items_discarded'],axis=1)
#dropping unrequired columns

In [ ]:
df_main_prob3.to_csv(r'C:\Users\Acer\Downloads\report3.csv')
#saving the dataframe as csv

# solution for report2

In [351]:
df_prob_2=df_main.iloc[0:,0:8]
#taking the required columns

In [354]:
df_prob_2=df_prob_2.drop(['Invoice_Id','Batch_Id','Production_Unit_Id'],axis=1)
#droping the columns which are in not use

In [356]:
df_prob_2=df_prob_2.sort_values(by='Customer_Id')
#sorting columns based on customer id

In [358]:
df_prob_2=df_prob_2.rename(columns={"rock_Items_Summary": "Rocks_bought", "scissor_Items_Summary": "Scissors_bought","paper_Items_Summary": "Paper_Bought"})
#changing column names for easy understanding

In [360]:
a=df_prob_2.iloc[0:,1]
#taking the required columns

In [362]:
a=pd.get_dummies(a)
#converting defective item column to numerical

In [364]:
df_prob_2=pd.concat([a,df_prob_2],join='outer',axis=1)
#combining two columns  after dummyfication

In [366]:
df_prob_2=df_prob_2.drop(['Defective_Item'],axis=1)
#droping the columns which are in not use

In [383]:
df_prob2_fin=df_prob_2.groupby('Customer_Id').sum()
#grouping other columns based on customer id

In [384]:
df_prob2_fin['Number_of_complaints']=df_prob2_fin['paper']+df_prob2_fin['rock']+df_prob2_fin['scissor']
#number of total complaint is the sum of individual item complaints

In [385]:
df_prob2_fin=df_prob2_fin.drop(['paper','rock','scissor'],axis=1)
#droping the columns which are in not use

In [526]:
b=df_prob2_fin.reset_index(level=0)
#changing the index to use in further steps

In [527]:
b=b.iloc[0:,0]
#taking index column separetaly

In [528]:
b=b.str.split(pat=('0'),expand=True)
b=b.iloc[0:,0]
#splitting index column name

In [529]:
b=b.str.split(pat=('1'),expand=True)
b=b.iloc[0:,0]
#splitting index column name

In [530]:
b=b.str.split(pat=('2'),expand=True)
b=b.iloc[0:,0]
#splitting index column name

In [531]:
b=b.str.split(pat=('3'),expand=True)
b=b.iloc[0:,0]
#splitting index column name

In [532]:
b=b.str.split(pat=('4'),expand=True)
b=b.iloc[0:,0]
#splitting index column name

In [533]:
b=b.str.split(pat=('5'),expand=True)
b=b.iloc[0:,0]
#splitting index column name

In [534]:
b=b.str.split(pat=('6'),expand=True)
b=b.iloc[0:,0]
#splitting index column name

In [535]:
b=b.str.split(pat=('7'),expand=True)
b=b.iloc[0:,0]
#splitting index column name

In [536]:
b=b.str.split(pat=('8'),expand=True)
b=b.iloc[0:,0]
#splitting index column name

In [537]:
b=b.str.split(pat=('9'),expand=True)
b=b.iloc[0:,0]
#splitting index column name

In [540]:
c=df_prob2_fin.reset_index(level=0)
#resetting index

In [561]:
df_prob2_final=pd.concat([b,c],join='outer',axis=1)
#joining two frames

In [562]:
df_prob2_final=df_prob2_final.drop(['Customer_Id'],axis=1)
#dropping unrequired column

In [563]:
df_prob2_final=df_prob2_final.groupby(0).sum()
#grouping by column number 0

In [564]:
df_prob2_final=df_prob2_final.rename_axis('Customer_Group')
#renaming column

In [566]:
columns_titles = ["Number_of_complaints","Rocks_bought","Scissors_bought","Paper_Bought"]
df_prob2_end=df_prob2_final.reindex(columns=columns_titles)
#resetting index

In [568]:
df_prob2_end.to_csv(r'C:\Users\Acer\Downloads\report2.csv')
#saving the frame as csv